In [ ]:
factors_r = ["SP500"] # "SP500" does not contain dividends
factors_d = ["DTB3"]

In [ ]:
exec(open("../helper-levels.py").read())

In [ ]:
width = 20 * 3

# Parse web


In [ ]:
import requests

In [ ]:
class Process:
  
  @staticmethod
  def filter(filter):
  
    operator, operands = filter
    
    result = {
      "operator": operator,
      "operands": [
          {"operator": operand[0], "operands": operand[1]} for operand in operands
      ],
    }
    
    return result
    
  # @staticmethod
  # def url(params):
  #   
  #   result = "?" + "&".join(f"{key}={value}" for key, value in params.items())
  #   
  #   return result
  
  @staticmethod
  def nest(df):
    
  	df = df.copy()
  	
  	for col in df.columns:
  	  
  		if df[col].apply(lambda x: isinstance(x, list)).all():
  			
  			status_df = df[col].apply(lambda x: all(isinstance(i, dict) for i in x)).all()
  			
  			if status_df:
  				
  				cols = set()
  				
  				for row in df[col]:
  					for item in row:
  					  
  						flattened_item = pd.json_normalize(item, sep = ".", max_level = None)
  						cols.update(flattened_item.columns)
  				
  				row_na = {key: None for key in cols}
  				
  				result_ls = []
  				
  				for row in df[col]:
  					
  					if not row:
  						result_ls.append(row_na)
  					else:
  					  
  						flattened_row = pd.json_normalize(row[0]).to_dict(orient = "records")[0]
  						result = {key: flattened_row.get(key, None) for key in cols}
  						
  						cols_na = cols - result.keys()
  						
  						for col_na in cols_na:
  							result[col_na] = None
  						
  						result_ls.append(result)
  				
  				result_df = pd.DataFrame(result_ls)
  				df = pd.concat([df.reset_index(drop = True), result_df], axis = 1)
  				
  				df.drop(columns = [col], inplace=True)
  				
  			else:
  				df[col] = None
  	
  	return df

In [ ]:
class Query:
  
  @staticmethod
  def create(filters = [("or", [("eq", ["region", "us"])])],
             top_operator = "and"):
    
    result = {
      "operator": top_operator,
      "operands": [Process.filter(filter) for filter in filters],
    }

    return result

In [ ]:
class Payload:
  
  @staticmethod
  def create(quote_type = "equity", query = Query.create(),
             size = 25, offset = 0,
             sort_field = None, sort_type = None,
             top_operator = "and"):
    
    result = {
      "includeFields": None,  # unable to modify the result
      "offset": offset,
      "query": query,
      "quoteType": quote_type,
      "size": size,
      "sortField": sort_field,
      "sortType": sort_type,
      "topOperator": top_operator,
    }
    
    return result

In [ ]:
class Session:
  
  @staticmethod
  def get():
    
    session = requests.Session()
    
    api_url = "https://query1.finance.yahoo.com/v1/test/getcrumb"
    
    headers = {
      "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    }
    
    session.headers.update(headers)
  
    response = session.get(api_url)
    
    crumb = response.text.strip()
    cookies = session.cookies.get_dict()
  
    result = {
      "handle": session,
      "crumb": crumb,
      "cookies": cookies
    }
    
    return result

In [ ]:
class Screen:
  
  @staticmethod
  def get(payload = Payload.create()):
  
    session = Session.get()
    crumb = session["crumb"]
    cookies = session["cookies"]
    handle = session["handle"]
  
    params = {
      "crumb": crumb,
      "lang": "en-US",
      "region": "US",
      "formatted": "true",
      "corsDomain": "finance.yahoo.com",
    }
  
    api_url = "https://query1.finance.yahoo.com/v1/finance/screener" # + Process.url(params)
  
    headers = {
      # "Content-Type": "application/json",
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    }
  
    max_size = 250
    size = payload["size"]
    offset = payload["offset"]
    
    result_cols = set()
    result_ls = []

    while size > 0:
  
      chunk_size = min(size, max_size)
      payload["size"] = chunk_size
      payload["offst"] = offset
  
      for key, value in cookies.items():
        handle.cookies.set(key, value)
  
      response = handle.post(api_url, params = params, json = payload, headers = headers)
  
      result = response.json()
      result_df = result["finance"]["result"][0]["quotes"]
  
      if (result_df is not None):
        
        result_df = pd.json_normalize(result_df)
        result_df = Process.nest(result_df)
        
        result_ls.append(result_df)
        result_cols.update(result_df.columns)
  
        size -= chunk_size
        offset += chunk_size
  
      else:
        size = 0
        
    result_cols = list(result_cols)
    
    for i in range(len(result_ls)):
      
      x = result_ls[i]
      cols_na = set(result_cols) - set(x.columns)
      
      for j in cols_na:
        x[j] = None
        
      result_ls[i] = x[result_cols]
    
    result = pd.concat(result_ls, ignore_index = True)
    
    return result

# Optimization


In [ ]:
import json
import cvxpy as cp

In [ ]:
def min_rss_optim(x, y):
    
    w = cp.Variable(x.shape[1])
    
    objective = cp.Minimize(cp.sum_squares(y - x @ w))
    
    constraints = [cp.sum(w) == 1, w >= 0, w <= 1]
    
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    return w.value

In [ ]:
filters = [("or", [("eq", ["categoryname", "Tactical Allocation"])])]
query = Query.create(filters)
payload = Payload.create(quote_type = "mutualfund", query = query, size = 250,
                         sort_field = "fundnetassets")
screen = Screen.get(payload)

In [ ]:
sorted_df = screen.sort_values(by = ["netAssets.raw", "firstTradeDateMilliseconds"])
tickers = sorted_df.loc[~sorted_df["netAssets.raw"].duplicated(), "symbol"].tolist()

In [ ]:
# allocations = ["AOK", "AOM", "AOR", "AOA"]
# tickers = tickers + allocations

In [ ]:
exec(open("../helper-prices.py").read())

In [ ]:
overlap_df = returns_df.rolling(scale["overlap"],min_periods = 1).mean()

# overlap_df = overlap_df.dropna()
overlap_x_df = overlap_df[factors]
overlap_y_df = overlap_df[tickers]
# overlap_z_df = overlap_df[allocations]

In [ ]:
def pnl(x):
    return np.nanprod(1 + x) - 1

In [ ]:
performance_df = returns_df.rolling(width, min_periods = 1).apply(pnl, raw = False)

In [ ]:
n_rows = overlap_df.shape[0]
result_ls = []
index_ls = []

# for i in range(width - 1, n_rows):
for i in range(n_rows - 1, n_rows):
  
  idx = range(max(i - width + 1, 0), i + 1)
  x_subset = overlap_x_df.iloc[idx]
  y_subset = overlap_y_df.iloc[idx]
  params_ls = []
  tickers_ls = []
  performance_ls = []
  
  # for j in [ticker for ticker in tickers if ticker not in allocations]:
  for j in tickers:
    
    idx = ~x_subset.isna().any(axis = 1) & ~y_subset[j].isna()
    x_complete = x_subset.loc[idx]
    y_complete = y_subset.loc[idx, j]
    
    if (x_complete.shape[0] > 0) and (y_complete.size > 0):
        
      params = min_rss_optim(x_complete.values, y_complete.values)
      params_ls.append(params)
      
      tickers_ls.append(j)
      
      performance_ls.append(performance_df[j].iloc[i])

  if params_ls:
    
    result = pd.DataFrame(params_ls, index = tickers_ls)
    result["performance"] = performance_ls
    
    result_ls.append(result)
    index_ls.append(overlap_x_df.index[i])

In [ ]:
# json.dump([x.to_dict() for x in result_ls], open("result_ls.json", "w"))
# json.dump([x.isoformat() for x in index_ls], open("index_ls.json", "w"))

# Performance


In [ ]:
# result_ls = [pd.DataFrame(x) for x in json.load(open("result_ls.json", "r"))]
# index_ls = [pd.Timestamp(x) for x in json.load(open("index_ls.json", "r"))]

In [ ]:
def quantile_cut(x):
  
  result = pd.qcut(
    -x,
    q = [0, 0.25, 0.5, 0.75, 1],
    labels = ["Q1", "Q2", "Q3", "Q4"]
  )
  
  return result

In [ ]:
n_rows = len(result_ls)
score_ls = []

for i in range(n_rows):
  
  score_df = pd.DataFrame(result_ls[i])
  score_df.columns = factors + ["performance"]
  
  score_df["date"] = index_ls[i]
  score_df["quantile"] = quantile_cut(score_df["performance"])
  
  score = score_df.groupby(["date", "quantile"]).agg(
    weight = (factors[0], "mean"),
    performance = ("performance", "mean")
  ).reset_index()
  
  overall = pd.DataFrame({
    "date": [index_ls[i]],
    "quantile": ["Overall"],
    "weight": [score_df[factors[0]].mean()],
    "performance": [score_df["performance"].mean()]
  })
  
  score = pd.concat([score, overall], ignore_index = True)
  
  score_ls.append(score)

In [ ]:
pd.concat(score_ls, ignore_index = True)